In [1]:
!pip install -qU pydicom
!pip install -qU torchinfo
!pip install -qU monai


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch_xla.core.xla_model as xm
xm.xla_device()

E0814 20:17:51.906138879      15 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2023-08-14T20:17:51.906121343+00:00", grpc_status:2}


device(type='xla', index=0)

In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import torch
import numpy as np
from glob import glob
import os
import pydicom
import matplotlib.pyplot as plt
import monai
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, ToTensor,ToNumpy
from torchinfo import summary
from monai.transforms import apply_transform
from tqdm import tqdm

In [4]:
image_size=160
BS=4
NW=4
N=len(os.listdir("/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images"))
EPOCHS=2
LR=0.001
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = xm.xla_device()

In [5]:
import matplotlib.animation as animation
from IPython.display import HTML

def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        new_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
        pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dcm)
    return pixel_array

def read_xray(path, fix_monochrome = True):
    dicom = pydicom.dcmread(path)
    data = standardize_pixel_array(dicom)
    data = data - np.min(data)
    data = data / (np.max(data) + 1e-5)
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = 1.0 - data
    return data

In [6]:
# # Grab every 10th DICOM file in the series
# dicom_files = sorted(glob('/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/10163/17557/*.dcm'))[::6]
# dicom_images = [read_xray(f) for f in dicom_files]

# fig, ax = plt.subplots()

# # plot the first frame and keep the plot object for later
# im = ax.imshow(dicom_images[0], cmap=plt.cm.bone)

# # create update function that iterates over your images
# def update(i):
#     im.set_array(dicom_images[i])

# ani = animation.FuncAnimation(fig, update, frames=range(len(dicom_images)), repeat=True)

# # Show the animation
# HTML(ani.to_jshtml())

In [7]:
val_transforms = Compose([ScaleIntensity(), Resize((image_size, image_size, image_size)), ToNumpy()])

In [19]:
class RSNA_DATASET(torch.utils.data.Dataset):
    def __init__(self,train=True,transforms=None):
        self.transforms=transforms
        self.train=train
        if self.train:
            self.patients=glob("/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/*")
            self.segmentations=glob("/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations/*")
            self.data=pd.read_csv("/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv")
        else:
            self.patients=glob("/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images/*")
        
    def __getitem__(self,index):
        #take one series only for now 
        series=glob(self.patients[index]+"/*")
        images=sorted(glob(series[0]+"/*"))
        images = np.stack([np.expand_dims(read_xray(image),2) for image in images], axis=2).transpose(3,0,1,2)
        patient=int(self.patients[index].split("/")[-1])
        
        if self.transforms is not None:
            images = apply_transform(self.transforms, images)
            
        if self.train:
            labels=self.data[self.data['patient_id']==patient]
            return images,torch.tensor(labels.iloc[:,1:3].to_numpy()).squeeze().float(),torch.tensor(labels.iloc[:,3:5].to_numpy()).squeeze().float(),torch.tensor(labels.iloc[:,5:8].to_numpy()).squeeze().float(),torch.tensor(labels.iloc[:,8:11].to_numpy()).squeeze().float(),torch.tensor(labels.iloc[:,11:14].to_numpy()).squeeze().float(),torch.tensor(labels.iloc[:,14:].to_numpy()).squeeze().float()
        else:
            return images,patient
        
    def __len__(self):
        return len(self.patients)

In [9]:
class RSNAResnet50(torch.nn.Module):
    def __init__(self,model):
        super(RSNAResnet50, self).__init__()
        self.resnet=torch.nn.Sequential(*list(model.children())[:-1]).to(device)
        
        for param in self.resnet.parameters():
            param.requires_grad = False
            
        self.fc=torch.nn.Sequential(torch.nn.Dropout(p=0.2, inplace=True),
                                    torch.nn.Linear(2048, 1024,bias=True),
                                    torch.nn.Dropout(p=0.2, inplace=True),
                                    torch.nn.Linear(1024, 512,bias=True),
                                    torch.nn.Dropout(p=0.2, inplace=True),
                                    torch.nn.Linear(512, 256,bias=True))
        
        self.bowel=torch.nn.Sequential(torch.nn.Linear(2048, 2,bias=True),torch.nn.Sigmoid()).to(device)
        
        self.ext=torch.nn.Sequential(torch.nn.Linear(2048, 2,bias=True),torch.nn.Sigmoid()).to(device)
        
        self.kidney=torch.nn.Sequential(torch.nn.Linear(2048, 3,bias=True),torch.nn.Softmax(dim=1)).to(device)
        
        self.liver=torch.nn.Sequential(torch.nn.Linear(2048, 3,bias=True),torch.nn.Softmax(dim=1)).to(device)
        
        self.spleen=torch.nn.Sequential(torch.nn.Linear(2048, 3,bias=True),torch.nn.Softmax(dim=1)).to(device)

    def forward(self, x):
        x = self.resnet(x.to(device))
        x=x.view(-1, 2048).to(device)
#       x = self.fc(x)
        bowel=self.bowel(x)
        ext=self.ext(x)
        kidney=self.kidney(x)
        liver=self.liver(x)
        spleen=self.spleen(x)
        
        return bowel,ext,kidney,liver,spleen

In [10]:
torch.manual_seed(42)
weights_dict = torch.load("/kaggle/input/weights/resnet_50.pth",map_location=torch.device('cpu'))
weights_dict = {k.replace('module.', ''): v for k, v in weights_dict['state_dict'].items()}
model=monai.networks.nets.resnet.resnet50(spatial_dims=3,n_input_channels=1)
model_dict = model.state_dict()
model_dict.update(weights_dict)
model.load_state_dict(model_dict)
model=RSNAResnet50(model).to(device)

In [11]:
# summary(model=model, 
#         input_size=(BS, 1, 160, 160, 160), 
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=15,
#         row_settings=["var_names"]
# )

In [12]:
train_ds=RSNA_DATASET(transforms=val_transforms)
train_dl=torch.utils.data.DataLoader(train_ds,batch_size=BS,num_workers=NW)

In [13]:
%%time
#losses
bowel_loss=torch.nn.BCELoss()
ext_loss=torch.nn.BCELoss()
kidney_loss=torch.nn.CrossEntropyLoss()
liver_loss=torch.nn.CrossEntropyLoss()
spleen_loss=torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LR)

for epoch in range(EPOCHS):
    print(epoch+1)
    for images,bowel,ext,kidney,liver,spleen,a_i in tqdm(train_dl):
        model.train()
        
        bowel_pred,ext_pred,kidney_pred,liver_pred,spleen_pred=model(images.to(device))
        loss=bowel_loss(bowel_pred,bowel.to(device))+ext_loss(ext_pred,ext.to(device))+kidney_loss(kidney_pred,kidney.to(device))+liver_loss(liver_pred,liver.to(device))+spleen_loss(spleen_pred,spleen.to(device))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        xm.mark_step()

1


100%|██████████| 787/787 [3:18:22<00:00, 15.12s/it]  


2


100%|██████████| 787/787 [3:01:46<00:00, 13.86s/it]  

CPU times: user 29min 2s, sys: 7min 40s, total: 36min 42s
Wall time: 6h 20min 8s


In [14]:
torch.save(model.state_dict(),'./resnet2.pt')

In [20]:
test_ds=RSNA_DATASET(transforms=val_transforms,train=False)
test_dl=torch.utils.data.DataLoader(test_ds,batch_size=1,num_workers=1)

In [23]:
%%time
data=[]
model.eval()
with torch.no_grad():
    for images,patient in test_dl:
        bowel_pred,ext_pred,kidney_pred,liver_pred,spleen_pred=model(images.to(device))
        row=[patient.item(),*bowel_pred.tolist()[0],*ext_pred.tolist()[0],*kidney_pred.tolist()[0],*liver_pred.tolist()[0],*spleen_pred.tolist()[0]]
        data.append(row)

sub = pd.DataFrame(data, columns=['patient_id', 'bowel_healthy', 'bowel_injury', 'extravasation_healthy',
       'extravasation_injury', 'kidney_healthy', 'kidney_low', 'kidney_high',
       'liver_healthy', 'liver_low', 'liver_high', 'spleen_healthy',
       'spleen_low', 'spleen_high'])
# print dataframe.
sub=sub.sort_values(by=['patient_id'])
sub

CPU times: user 152 ms, sys: 659 ms, total: 811 ms
Wall time: 6.05 s


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
2,48843,0.981519,0.018130,0.924235,0.075731,0.999810,0.000090,0.000100,0.999806,0.000108,0.000086,0.999803,0.000103,0.000094
1,50046,0.971639,0.028099,0.904721,0.094294,0.999231,0.000364,0.000406,0.999203,0.000440,0.000357,0.999217,0.000404,0.000379
0,63706,0.993861,0.006135,0.954793,0.046935,0.999984,0.000007,0.000009,0.999984,0.000009,0.000007,0.999983,0.000009,0.000008
